**Actividad 05/11/2025**

Ivo Giuliano Cappetto

Martin Sanchez

Nicolas Fernandez

Santiago Luna

1 - **Resumen de arquitecturas observadas:**

El proyecto “Programa un coche Arduino con Inteligencia Artificial” presenta dos tipos de arquitectura:

**A) Arquitectura de software (red neuronal):**

Red neuronal feedforward con propagación hacia adelante.

Compuesta por 3 capas:

Capa de entrada: 2 neuronas (distancia, dirección).

Capa oculta: 3 neuronas con función de activación tanh.

Capa de salida: 4 neuronas (una por motor).

Entrenamiento supervisado en Python mediante backpropagation.

Los pesos entrenados se exportan al Arduino para inferencia.

**B) Arquitectura de hardware (Arduino):**

Sensores ultrasónicos para medir distancia.

Servo motor que barre de izquierda a derecha.

Módulo controlador L298N para los 4 motores.

Flujo de datos: sensores → red neuronal → motores.

Conclusión:
Es un sistema híbrido (software + hardware) donde el aprendizaje ocurre en Python y la inferencia en Arduino.

**2 -  Enfoques de resolución de problemas aplicados:**

| Enfoque | Descripción |
|----------|--------------|
| **Aprendizaje supervisado** | Se definen entradas/salidas esperadas (tabla de verdad) para que la red aprenda el comportamiento del coche. |
| **Normalización** | Escala de −1 a 1 para que la activación `tanh` funcione correctamente. |
| **Entrenamiento iterativo** | Ajuste de pesos durante miles de épocas para minimizar el error. |
| **Generalización** | La red aprende patrones sin reglas manuales “if/else”. |
| **Despliegue embebido** | Los pesos se copian al Arduino Uno para ejecutar en tiempo real. |
| **Escalabilidad** | Se pueden añadir sensores o salidas reentrenando la red. |


**3 - Entrenar la red neuronal en Google Colab:**

**Configurar entorno:**

In [18]:
import numpy as np
import matplotlib.pyplot as plt

print("Entorno listo para entrenar la red neuronal")


Entorno listo para entrenar la red neuronal


**Clase NeuralNetwork:**

In [19]:
class NeuralNetwork:
    def __init__(self, layers):
        self.layers = layers
        # Inicialización de pesos aleatorios (-1 a 1)
        self.weights = [2 * np.random.random((layers[i] + 1, layers[i + 1])) - 1
                        for i in range(len(layers) - 1)]

    def tanh(self, x):
        return np.tanh(x)

    def tanh_deriv(self, x):
        return 1.0 - np.tanh(x)**2

    def fit(self, X, y, lr=0.03, epochs=40000):
        # Agregar columna de bias
        X = np.hstack([X, np.ones((X.shape[0], 1))])
        for k in range(epochs):
            i = np.random.randint(X.shape[0])
            a = [X[i]]
            # Propagación hacia adelante
            for l in range(len(self.weights)):
                a.append(self.tanh(np.dot(a[l], self.weights[l])))
            # Error y retropropagación
            error = y[i] - a[-1]
            deltas = [error * self.tanh_deriv(a[-1])]
            for l in range(len(a) - 2, 0, -1):
                deltas.append(deltas[-1].dot(self.weights[l].T) * self.tanh_deriv(a[l]))
            deltas.reverse()
            # Actualización de pesos
            for j in range(len(self.weights)):
                self.weights[j] += lr * np.atleast_2d(a[j]).T.dot(np.atleast_2d(deltas[j]))

    def predict(self, X):
        X = np.hstack([X, np.ones((X.shape[0], 1))])
        a = X
        for l in range(len(self.weights)):
            a = self.tanh(np.dot(a, self.weights[l]))
        return a


**Datos del coche Arduino y entrenamiento:**

In [22]:
import numpy as np

class NeuralNetwork:
    def __init__(self, layers):
        self.layers = layers
        self.weights = []
        # Inicializar pesos (incluyendo bias)
        for i in range(len(layers) - 1):
            w = 2 * np.random.random((layers[i] + 1, layers[i + 1])) - 1
            self.weights.append(w)

    def tanh(self, x):
        return np.tanh(x)

    def tanh_deriv(self, x):
        return 1.0 - np.tanh(x)**2

    def fit(self, X, y, lr=0.03, epochs=40000):
        # Añadir columna de bias a las entradas
        X = np.concatenate((X, np.ones((X.shape[0], 1))), axis=1)

        for k in range(epochs):
            i = np.random.randint(X.shape[0])
            a = [X[i]]

            # --- Forward propagation ---
            for l in range(len(self.weights)):
                z = np.dot(a[l], self.weights[l])
                if l < len(self.weights) - 1:
                    z = np.concatenate(([1], z))  # añadir bias solo en capas ocultas
                a.append(self.tanh(z))

            # --- Backpropagation ---
            error = y[i] - a[-1]
            deltas = [error * self.tanh_deriv(a[-1])]

            for l in range(len(a) - 2, 0, -1):
                delta = deltas[-1].dot(self.weights[l].T) * self.tanh_deriv(a[l])
                deltas.append(delta[1:])  # quitar bias
            deltas.reverse()

            # --- Actualización de pesos ---
            for j in range(len(self.weights)):
                layer = np.atleast_2d(a[j])
                delta = np.atleast_2d(deltas[j])
                self.weights[j] += lr * layer.T.dot(delta)

    def predict(self, X):
        # Agregar bias solo una vez al inicio
        X = np.concatenate((X, np.ones((X.shape[0], 1))), axis=1)
        a = X

        # --- Propagación sin añadir bias de más ---
        for l in range(len(self.weights)):
            z = np.dot(a, self.weights[l])
            a = self.tanh(z)

            # Añadir bias sólo si no estamos en la última capa
            if l < len(self.weights) - 1:
                a = np.concatenate((np.ones((a.shape[0], 1)), a), axis=1)
        return a


**Ver resultados (predicciones)**

In [23]:
# Datos de entrenamiento
X = np.array([
    [-1, -1],
    [-1,  0],
    [-1,  1],
    [ 0, -1],
    [ 0,  0],
    [ 0,  1],
    [ 1, -1],
    [ 1,  0],
    [ 1,  1]
])

y = np.array([
    [1, 0, 0, 1],
    [1, 0, 1, 0],
    [0, 1, 1, 0],
    [1, 0, 0, 1],
    [1, 0, 1, 0],
    [0, 1, 1, 0],
    [1, 0, 0, 1],
    [1, 0, 1, 0],
    [0, 1, 1, 0]
])

# Crear y entrenar la red
nn = NeuralNetwork([2, 3, 4])
nn.fit(X, y, lr=0.03, epochs=40000)

# Predicciones
predicciones = np.round(nn.predict(X), 2)
print("🔹 Predicciones:")
print(predicciones)


🔹 Predicciones:
[[1.   0.18 0.36 1.  ]
 [1.   0.18 1.   0.3 ]
 [0.3  1.   1.   0.3 ]
 [1.   0.18 0.36 1.  ]
 [1.   0.18 1.   0.3 ]
 [0.3  1.   1.   0.3 ]
 [1.   0.18 0.36 1.  ]
 [1.   0.18 1.   0.3 ]
 [0.3  1.   1.   0.3 ]]


**Valores:**

In [24]:
for i, w in enumerate(nn.weights):
    print(f"\nPesos capa {i+1}:\n")
    print(np.round(w, 3).tolist())



Pesos capa 1:

[[0.0, 0.0, 0.0], [-0.123, 2.352, -2.79], [-2.153, 0.998, 0.839]]

Pesos capa 2:

[[1.275, 0.773, 1.572, 1.313], [-1.265, -1.36, -1.156, -1.0], [0.002, -0.446, 2.157, -2.299], [2.304, -2.296, -0.427, -0.327]]
